In [295]:
import pandas as pd
import numpy as np
import re

In [296]:
df1 = pd.read_csv('github/Scraped_Review_Data/beerreview1_13.csv')
df2 = pd.read_csv('github/Scraped_Review_Data/beerreview14_26.csv')
df3 = pd.read_csv('github/Scraped_Review_Data/beerreview27_39.csv')
df4 = pd.read_csv('github/Scraped_Review_Data/beerreview40_51.csv')

In [297]:
df = pd.concat([df1, df2, df3, df4], axis = 0, ignore_index = True)

In [298]:
df.head()

,palate,taste,appearance,review,overall,aroma,user_rating,user_info,state,beer_name,user_name
0,4,7,3,"3.7| Bottle (12 oz). Shared by JulienHuxley, t...",15,8,3.7,"- Quebec, CANADA - JUL 31, 2015",Alabama,Avondale Barrel Aged Brett Saison,tom10101
1,4,8,3,Bottle shared with Tom. Hazy gold-amber pour w...,15,8,3.8,"- Trois-Rivi̬res, Quebec, CANADA - JUL 24, 2015",Alabama,Avondale Barrel Aged Brett Saison,JulienHuxley
2,4,6,3,Bottle thanks to Seth. Pours a cloudy yellow o...,16,7,3.6,"- Houston, Texas, USA - FEB 26, 2015",Alabama,Avondale Barrel Aged Brett Saison,djd07
3,4,8,5,"Excellent taste, good flaver, good body, stron...",17,9,4.3,"- - DEC 28, 2014",Alabama,Avondale Barrel Aged Brett Saison,Justosj76
4,4,9,3,Plurged and went for their barrel aged series ...,17,8,4.1,"- Atlanta, Georgia, USA - AUG 13, 2014",Alabama,Avondale Barrel Aged Brett Saison,Westmeister


In [299]:
df = df[['user_name', 'user_info', 'state', 'beer_name', 'overall', 'user_rating', 'aroma',
        'appearance', 'palate', 'taste']]   # sort the col and get rid of review col

In [300]:
df.head()

,user_name,user_info,state,beer_name,overall,user_rating,aroma,appearance,palate,taste
0,tom10101,"- Quebec, CANADA - JUL 31, 2015",Alabama,Avondale Barrel Aged Brett Saison,15,3.7,8,3,4,7
1,JulienHuxley,"- Trois-Rivi̬res, Quebec, CANADA - JUL 24, 2015",Alabama,Avondale Barrel Aged Brett Saison,15,3.8,8,3,4,8
2,djd07,"- Houston, Texas, USA - FEB 26, 2015",Alabama,Avondale Barrel Aged Brett Saison,16,3.6,7,3,4,6
3,Justosj76,"- - DEC 28, 2014",Alabama,Avondale Barrel Aged Brett Saison,17,4.3,9,5,4,8
4,Westmeister,"- Atlanta, Georgia, USA - AUG 13, 2014",Alabama,Avondale Barrel Aged Brett Saison,17,4.1,8,3,4,9


In [301]:
df[df['user_name'].isnull()].index.tolist()   # locate the user with missing name

[50128]

In [302]:
df.loc[50128, :]  # verify this location

user_name                                                    NaN
user_info               - Kennesaw, Georgia, USA - SEP 25, 2008 
state                                                   Delaware
beer_name      Dogfish Head World Wide Stout 2001/2003-Presen...
overall                                                       17
user_rating                                                  4.3
aroma                                                          8
appearance                                                     5
palate                                                         4
taste                                                          9
Name: 50128, dtype: object

In [303]:
df.drop(50128, inplace=True)   # remove this obs

In [304]:
df[df['user_name'].isnull()].index.tolist()  # verify

[]

In [305]:
pd.DataFrame(df.isnull().sum()).T  # no missing value anymore

,user_name,user_info,state,beer_name,overall,user_rating,aroma,appearance,palate,taste
0,0,0,0,0,0,0,0,0,0,0


In [306]:
pd.DataFrame(map(lambda x: type(x), list(df.iloc[0])), index=df.columns).T   # data type for each column

,user_name,user_info,state,beer_name,overall,user_rating,aroma,appearance,palate,taste
0,<type 'str'>,<type 'str'>,<type 'str'>,<type 'str'>,<type 'numpy.int64'>,<type 'numpy.float64'>,<type 'numpy.int64'>,<type 'numpy.int64'>,<type 'numpy.int64'>,<type 'numpy.int64'>


In [307]:
df.shape

(276693, 10)

## Clean by Column

### 1. user_name

In [308]:
un = df['user_name']
print len(un)
clean_name = lambda x: ''.join(e for e in re.sub('\([^)]*\)', '', x) if e.isalnum())
un2 = map(lambda x: clean_name(x), un)  # clean user_name col
print len(un2)

276693
276693


In [309]:
un2.count('')

0

In [310]:
df['user_name'] = un2

### 2. user_info

For this column, only date is extracted for now.

In [313]:
ui = df['user_info']

In [314]:
ui[:5]

0                     - Quebec, CANADA - JUL 31, 2015
1     - Trois-Rivi̬res, Quebec, CANADA - JUL 24, 2015
2                - Houston, Texas, USA - FEB 26, 2015
3                                   -  - DEC 28, 2014
4              - Atlanta, Georgia, USA - AUG 13, 2014
Name: user_info, dtype: object

In [315]:
find_date = lambda x: re.search('[A-Z]{3} \d{1,2}, \d{4}$', x)
user_date = map(lambda x: find_date(x).group() if find_date(x) else np.nan, ui)   # extract date str from the user_info

In [316]:
user_date.count(np.nan)   # count missing values after extracting date

6759

In [317]:
# verify the length of the str in user_date list. Possible values are 11 and 12.
pd.Series(map(lambda x: len(x), [item for item in user_date if item is not np.nan])).unique()

array([12, 11], dtype=int64)

In [318]:
user_date = pd.to_datetime(user_date, errors='ignore', format='%b %d, %Y')

In [319]:
df['user_date'] = user_date

In [ ]:
df = df[['user_name', 'user_date', 'state', 'beer_name', 'overall', 'taste', 'aroma',
        'appearance', 'palate', 'user_rating']]   # sort the col and get rid of user_info col

### 3. state

In [ ]:
st = df['state']

In [ ]:
len(st.unique())   # state variable is very clean

### 4. beer_name

In [ ]:
bn = df['beer_name']

In [ ]:
df['beer_name'] = map(lambda x: x.decode('utf-8', 'ignore').encode('utf-8', 'ignore'), bn)

### 5. overall, taste, aroma, appearance, palate and user_rating (clean)

In [ ]:
df.head()

In [ ]:
pd.DataFrame(df.isnull().sum()).T  # no missing value anymore

In [ ]:
df.to_csv('user_info_rating_proc.csv', index=False)   # export df